In [13]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

In [14]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [15]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Barge1000',
    "route": None,
    "geometry": None,
    "V_g_ave": 1.2,  # m/s
    "V_s":1.2,
    "type": None,
    "B": 11,
    "L": 60,
    "H_e": None, 
    "H_f": None, 
    "T": 3.45,
    "safety_margin": 0.3,                                      # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": True,                                            #  if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Tanker",                                    #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "V_g_profile":False,                                        # if use the V_g_profile to determine V_w, set to True, otherwise (use the V_g_ave to determine V_w) set to False.
    "P_installed": 750, # kW  
    "P_tot_given": None, # kW
    "bulbous_bow": False,                                       # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": False,
    "sailing_upstream": False,
    "consider_passive_rudder_resistance": False,
    "consider_wind_influence":False,
    "U_wind": 10, # m/s
    "rel_winddir": 10, #degree
    "load": 'loaded',
    "loaded_depth": 3.45 ,
    "unloaded_depth": 1.7,
    "Height_ship": 4,
    "P_hotel_perc": 0,
    "P_hotel": 0 , # None: calculate P_hotel from percentage
    "x": 2,# number of propellers
    "L_w": 3.0 ,
    "C_B":0.8, 
    "C_year": 2000,
}             

In [16]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

In [17]:
h_0 = [18]    
V_s = [1.2,10,11,12]
L = [50,65,75,85,91]            
B = [10,11,14,15,19]   
T = [3.4,3.18,3.75,2.15,3.35,2.35,6.5,5.7,6.8,2.4]
P_installed = [900,1570,2000,3000,9400]

In [18]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(h_0, V_s, L,B,T,P_installed))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"h_0": item[0], "V_s": item[1], "L": item[2], "B": item[3], "T": item[4], "P_installed": item[5]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.tail()

,h_0,V_s,L,B,T,P_installed
4995,18,12.0,91,19,2.4,900
4996,18,12.0,91,19,2.4,1570
4997,18,12.0,91,19,2.4,2000
4998,18,12.0,91,19,2.4,3000
4999,18,12.0,91,19,2.4,9400


In [19]:
h_0 = 18
V_s = 1.2
V_g_ave = 1.2
results =[]
U_wind = 0
rel_winddir = 0
for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    h_0 = row['h_0']
    data_vessel_i = data_vessel.copy()
    data_vessel_i['h_0'] = h_0
    vessel = TransportResource(**data_vessel_i)
    
    V_s = row['V_s']
    L = row['L']
    B = row['B']
    T = row['T']
    P_installed = row['P_installed']

    vessel.calculate_properties() # L is used here in the computation of L_R
   # print(h_0)
    R_f = vessel.calculate_frictional_resistance(V_s, h_0) 
    R_f_one_k1 = vessel.calculate_viscous_resistance()
    R_APP = vessel.calculate_appendage_resistance(V_s)
    R_W = vessel.calculate_wave_resistance(V_s, h_0)
    R_res = vessel.calculate_residual_resistance(V_s, h_0)
    R_wind = vessel.calculate_wind_resistance(U_wind,rel_winddir)
    R_T = vessel.calculate_total_resistance(V_s, h_0, rel_winddir)
    P_propulsion, P_tot, P_given = vessel.calculate_total_power_required(V_s, h_0)
    vessel.emission_factors_general()
    vessel.correction_factors(V_s,h_0)
    vessel.calculate_emission_factors_total(V_s,h_0)
 #   Fuel_g_m = vessel.calculate_fuel_use_g_m(V_s)
    [emission_g_m_CO2,emission_g_m_PM10,emission_g_m_NOX] = vessel.calculate_emission_rates_g_m(V_s)
   # print(vessel.A_xv)
    result = {}
    result.update(row)
    result['P_installed'] = vessel.P_installed
    result['R_f_one_k1'] = R_f_one_k1
    result['R_APP'] = R_APP
    result['R_W'] = R_W
    result['R_res'] = R_res
    result['R_T'] = R_T
    result['R_wind'] = R_wind
    result['P_tot'] = P_tot
   # result['Fuel_g_km'] = Fuel_g_m * 1000
    result['emission_g_km_CO2'] = emission_g_m_CO2 * 1000
    result['emission_g_km_PM10'] = emission_g_m_PM10 * 1000
    result['emission_g_km_NOX'] = emission_g_m_NOX * 1000
    results.append(result)

5000it [01:13, 67.78it/s] 


In [20]:
plot_df = pd.DataFrame(results)


# convert from meters per second to km per hour
ms_to_kmh = 3.6
plot_df['V_s_km'] = plot_df['h_0'] * ms_to_kmh

plot_df.head(100)

,h_0,V_s,L,B,T,P_installed,R_f_one_k1,R_APP,R_W,R_res,R_T,R_wind,P_tot,emission_g_km_CO2,emission_g_km_PM10,emission_g_km_NOX,V_s_km
0,18.0,1.2,50.0,10.0,3.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
1,18.0,1.2,50.0,10.0,3.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
2,18.0,1.2,50.0,10.0,3.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
3,18.0,1.2,50.0,10.0,3.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
4,18.0,1.2,50.0,10.0,3.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,18.0,1.2,50.0,11.0,2.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
96,18.0,1.2,50.0,11.0,2.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
97,18.0,1.2,50.0,11.0,2.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
98,18.0,1.2,50.0,11.0,2.4,750,1.86017,0.176243,8.896020e-10,1.389105,3.425518,0,6.851036,1288.533891,1.16087,27.280443,64.8
